In [15]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import scipy as cp
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import yfinance as yf
import random
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error
from datetime import timedelta
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import StandardScaler
import time
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from scipy.optimize import minimize
ESG_df=pd.read_excel("/content/drive/My Drive/UW/Actsc991/ESG 2022.xlsx").iloc[1:,1:3]
ESG_tickers=list(ESG_df["Symbol"])

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
# Define the start and end date for the past ten years
end_date = pd.Timestamp.now()
start_date = end_date - pd.DateOffset(years=10)
# Function to get daily stock data for a given ticker
def get_stock_data(ticker, start_date, end_date):
    try:
        stock_data = yf.download(ticker, start=start_date, end=end_date)
        return stock_data
    except Exception as e:
        print(f"Failed to download data for {ticker}: {e}")
        return None
ESG_stock_data = {}

# Keep trying until we get data for 30 tickers
# Keep trying until we get data for 30 tickers
while len(ESG_stock_data) < 30:
    # Randomly sample 1 ticker from the list
    random_ticker = np.random.choice(ESG_tickers, size=1, replace=False)[0]

    # If we already have this ticker, skip to the next iteration
    if random_ticker in ESG_stock_data:
        continue

    data = get_stock_data(random_ticker, start_date=start_date, end_date=end_date)

    # Validate that the data contains at least 10 years of data
    # Assuming at least 250 trading days in a year
    if data is not None and len(data) >= 250 * 10:
        ESG_stock_data[random_ticker] = data
    else:
        print(f"Insufficient data for {random_ticker}, trying another ticker.")

print("Stock data downloaded successfully for tickers:", list(ESG_stock_data.keys()))

print("Stock data downloaded successfully for tickers:", list(ESG_stock_data.keys()))
stock_names = [key for key in ESG_stock_data]
ESG_result_dict = {}
for dataset_name, dataset in ESG_stock_data.items():
    # Assuming your data is sorted by date, you can calculate the logarithmic return
    dataset['rt'] = np.log(dataset['Close']) - np.log(dataset['Close'].shift(1))
    # Calculate the one-day lagged logarithmic return
    dataset['rt-1'] = dataset['rt'].shift(1)
    # Calculate the two-day lagged logarithmic return
    dataset['rt-2'] = np.log(dataset['Open'].shift(2)) - np.log(dataset['Close'].shift(1))
    # Calculate the three-day lagged logarithmic return
    dataset['rt-3'] = np.log(dataset['Open'].shift(3)) - np.log(dataset['Close'].shift(1))
    dataset['ct'] = np.log(dataset['Close']) - np.log(dataset['Open'])
    dataset['ct-1'] = np.log(dataset['Close']) - np.log(dataset['Open'].shift(1))
    dataset['ot'] = np.log(dataset['Open']) - np.log(dataset['Close'].shift(1))
    dataset['ht'] = np.log(dataset['High']) - np.log(dataset['Open'])
    dataset['ht-1'] = np.log(dataset['High'].shift(1)) - np.log(dataset['Open'].shift(1))
    dataset['ht-2'] = np.log(dataset['High'].shift(2)) - np.log(dataset['Open'].shift(2))
    dataset['ht-2'] = np.log(dataset['High'].shift(3)) - np.log(dataset['Open'].shift(3))
    dataset['lt'] = np.log(dataset['Low']) - np.log(dataset['Open'])
    dataset['lt-1'] = np.log(dataset['Low'].shift(1)) - np.log(dataset['Open'].shift(1))
    dataset['lt-2'] = np.log(dataset['Low'].shift(2)) - np.log(dataset['Open'].shift(2))
    dataset['lt-3'] = np.log(dataset['Low'].shift(3)) - np.log(dataset['Open'].shift(3))

    close_prices = dataset['Close']
    high_prices = dataset['High']
    low_prices = dataset['Low']
    prev_close = np.roll(close_prices, 1)
    true_range = np.maximum(
        np.abs(high_prices - low_prices),
        np.abs(prev_close - high_prices),
        np.abs(prev_close - low_prices)
    )
    dataset=dataset.reset_index()
    dataset["Date"]=pd.to_datetime(dataset["Date"])
    result = dataset.dropna()
    ESG_result_dict[dataset_name] = result
def create_rolling_windows(stock_df, num_windows=7, train_years=3, test_years=1):
    rolling_windows = []
    start_date = stock_df['Date'].min()

    for _ in range(num_windows):
        train_end_date = start_date + pd.DateOffset(years=train_years)
        test_end_date = train_end_date + pd.DateOffset(years=test_years)

        if test_end_date > stock_df['Date'].max():
            break  # Stop if we don't have enough data for the next window

        # Extract training and testing sets
        training_set = stock_df[(stock_df['Date'] >= start_date) & (stock_df['Date'] < train_end_date)]
        testing_set = stock_df[(stock_df['Date'] >= train_end_date) & (stock_df['Date'] < test_end_date)]

        # Store the pairs
        rolling_windows.append((training_set, testing_set))

        # Move the start date by one year for the next window
        start_date += pd.DateOffset(years=1)

    return rolling_windows

# Create a new dictionary with rolling window pairs for each stock
ESG_Train_Test_Dict = {stock_name: create_rolling_windows(stock_df) for stock_name, stock_df in ESG_result_dict.items()}
def calculate_averages(data):
    grouped_data = {}

    # 遍历每个股票
    for stock in data:
        # 遍历每个年份
        for year_data in data[stock]:
            for year, records in year_data.items():
                # 初始化每年的总数和计数器
                totals = {'mse': 0, 'rmse': 0, 'mae': 0, 'hit_rate': 0}
                count = len(records)

                # 累加每个指标
                for record in records:
                    for key in totals:
                        totals[key] += record[key]

                # 计算平均值并存储在 grouped_data 中
                if year not in grouped_data:
                    grouped_data[year] = {key: 0 for key in totals}
                for key in totals:
                    grouped_data[year][key] += totals[key] / count

    # 对每个年份的指标平均值进行平均化
    for year in grouped_data:
        for key in grouped_data[year]:
            grouped_data[year][key] /= len(data)

    return grouped_data

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


Insufficient data for KEYS, trying another ticker.


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


Insufficient data for VVV, trying another ticker.


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

Stock data downloaded successfully for tickers: ['COP', 'MPC', 'XEL', 'ALB', 'DTE', 'LLY', 'CMI', 'PKG', 'CVX', 'CMG', 'PSA', 'LNG', 'HAL', 'TMO', 'PEP', 'EPAM', 'MRO', 'BG', 'CDNS', 'AES', 'DOX', 'VRSK', 'DECK', 'ADI', 'LECO', 'KBR', 'AMT', 'BR', 'HES', 'D']
Stock data downloaded successfully for tickers: ['COP', 'MPC', 'XEL', 'ALB', 'DTE', 'LLY', 'CMI', 'PKG', 'CVX', 'CMG', 'PSA', 'LNG', 'HAL', 'TMO', 'PEP', 'EPAM', 'MRO', 'BG', 'CDNS', 'AES', 'DOX', 'VRSK', 'DECK', 'ADI', 'LECO', 'KBR', 'AMT', 'BR', 'HES', 'D']


In [17]:
start_time = time.time()
target_column = 'rt'
rf_return={}
# Initialize a dictionary to store the best models and their performance for each stock
RF_performance = {}

for stock_name, windows in ESG_Train_Test_Dict.items():
    error_year=[]
    ret=[]
    # Initialize a list to store the best models and their metrics for the current stock
    stock_best_models = []
    param_grid = {
    'max_depth': [15],
    'min_samples_split': [15],
    'min_samples_leaf': [15],
    'n_estimators': [500]}
    # Loop over each pair of training/testing set
    years=[2017,2018,2019,2020,2021,2022]
    count=0
    for  i in windows:
        train_df, test_df= i
        # Prepare the training data
        X_train = train_df.iloc[:,7:].drop(columns=[target_column])
        y_train = train_df.iloc[:,7:][target_column]

        # Prepare the testing data
        X_test = test_df.iloc[:,7:].drop(columns=[target_column])
        y_test = test_df.iloc[:,7:][target_column]

        # Initialize the GridSearchCV object
        rf = RandomForestRegressor(random_state=0)
        grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, n_jobs=-1, scoring='neg_mean_squared_error')

        # Fit the grid search to the data
        grid_search.fit(X_train, y_train)

        # Get the best estimator
        best_rf = grid_search.best_estimator_

        # Predict on the testing set with the best estimator
        y_pred = best_rf.predict(X_test)


        ret.append(y_pred)


        # Calculate the performance metrics
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(y_test, y_pred)

        # Calculate Hit Rate
        # Assuming the target column is the price and we are interested in the direction of the change
        direction_pred = np.sign(np.diff(y_pred))
        direction_true = np.sign(np.diff(y_test.values))
        hit_rate = np.mean(direction_pred == direction_true)

        # Store the best model and its performance metrics
        stock_best_models.append({
            'model': best_rf,
            'mse': mse,
            'rmse': rmse,
            'mae': mae,
            'hit_rate': hit_rate
        })
        error_year.append({years[count]:stock_best_models})
        print("the Prediction of year "+str(years[count])+" of " +stock_name+" is done")
        print(str(years[count])+":")
        print(stock_best_models)
        count+=1
    # Add the best models and their performance to the dictionary
    RF_performance[stock_name] = error_year
    rf_return[stock_name]=ret
end_time = time.time()
print(f"Execution time: {end_time - start_time} seconds")

the Prediction of year 2017 of COP is done
2017:
[{'model': RandomForestRegressor(max_depth=15, min_samples_leaf=15, min_samples_split=15,
                      n_estimators=500, random_state=0), 'mse': 3.772899866802114e-06, 'rmse': 0.0019423953940436829, 'mae': 0.0013104993364071913, 'hit_rate': 0.98}]
the Prediction of year 2018 of COP is done
2018:
[{'model': RandomForestRegressor(max_depth=15, min_samples_leaf=15, min_samples_split=15,
                      n_estimators=500, random_state=0), 'mse': 3.772899866802114e-06, 'rmse': 0.0019423953940436829, 'mae': 0.0013104993364071913, 'hit_rate': 0.98}, {'model': RandomForestRegressor(max_depth=15, min_samples_leaf=15, min_samples_split=15,
                      n_estimators=500, random_state=0), 'mse': 2.611002014636323e-05, 'rmse': 0.005109796487763796, 'mae': 0.0030951211921262708, 'hit_rate': 0.952}]
the Prediction of year 2019 of COP is done
2019:
[{'model': RandomForestRegressor(max_depth=15, min_samples_leaf=15, min_samples_spl

In [18]:

start_time = time.time()
target_column = 'rt'

# Initialize a dictionary to store the best models and their performance for each stock
LSTM_performance = {}
LSTM_stock_return = {}

# Define LSTM model architecture with new parameters
def create_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(128, return_sequences=True, input_shape=input_shape, dropout=0.4))
    model.add(LSTM(128, return_sequences=False, dropout=0.4))
    model.add(Dense(25))
    model.add(Dense(1))
    model.compile(optimizer="Adam", loss='mean_squared_error')
    return model

# Loop over each stock
for stock_name, windows in ESG_Train_Test_Dict.items():
    stock_best_models = []
    ret = []
    years = [2017, 2018, 2019, 2020, 2021, 2022]
    for year, (train_df, test_df) in zip(years, windows):
        # Standardize the data
        scaler_X = StandardScaler()
        scaler_y = StandardScaler()
        X_train_scaled = scaler_X.fit_transform(train_df.iloc[:, 7:].drop(columns=[target_column]))
        y_train_scaled = scaler_y.fit_transform(train_df[[target_column]])
        X_test_scaled = scaler_X.transform(test_df.iloc[:, 7:].drop(columns=[target_column]))
        y_test_scaled = scaler_y.transform(test_df[[target_column]])

        # Reshape data for LSTM model
        X_train_scaled = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
        X_test_scaled = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

        # Define and fit the LSTM model
        lstm_model = create_lstm_model(X_train_scaled.shape[1:])
        lstm_model.fit(X_train_scaled, y_train_scaled, batch_size=32, epochs=64)

        # Predict on the test set
        y_pred_scaled = lstm_model.predict(X_test_scaled)

        # Inverse transform predictions and actual values to original scale
        y_pred = scaler_y.inverse_transform(y_pred_scaled).flatten()
        y_test = scaler_y.inverse_transform(y_test_scaled).flatten()

        ret.append(y_pred)

        # Calculate performance metrics
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(y_test, y_pred)

        # Calculate Hit Rate
        direction_pred = np.sign(np.diff(y_pred))
        direction_true = np.sign(np.diff(y_test))
        hit_rate = np.mean(direction_pred == direction_true)

        # Store the best model and its performance metrics
        stock_best_models.append({
            'model': lstm_model,
            'mse': mse,
            'rmse': rmse,
            'mae': mae,
            'hit_rate': hit_rate
        })

        print(f"Prediction for year {year} of {stock_name} is done.")
        print(f"{year}: {stock_best_models[-1]}")

    # Add the best models and their performance to the dictionary
    LSTM_performance[stock_name] = stock_best_models
    LSTM_stock_return[stock_name] = ret

end_time = time.time()
print(f"Execution time: {end_time - start_time} seconds")

Streaming output truncated to the last 5000 lines.
Epoch 56/64
24/24 [==============================] - 0s 6ms/step - loss: 0.2303
Epoch 57/64
24/24 [==============================] - 0s 6ms/step - loss: 0.2333
Epoch 58/64
24/24 [==============================] - 0s 6ms/step - loss: 0.2449
Epoch 59/64
24/24 [==============================] - 0s 6ms/step - loss: 0.2051
Epoch 60/64
24/24 [==============================] - 0s 6ms/step - loss: 0.2239
Epoch 61/64
24/24 [==============================] - 0s 6ms/step - loss: 0.2492
Epoch 62/64
24/24 [==============================] - 0s 6ms/step - loss: 0.2218
Epoch 63/64
24/24 [==============================] - 0s 6ms/step - loss: 0.1994
Epoch 64/64
8/8 [==============================] - 1s 3ms/step
Prediction for year 2020 of ADI is done.
2020: {'model': <keras.src.engine.sequential.Sequential object at 0x7bd30c8972b0>, 'mse': 4.795943320801171e-05, 'rmse': 0.006925274955408753, 'mae': 0.005330816611994366, 'hit_rate': 0.952}
Epoch 1/64
24/

In [19]:
start_time = time.time()
target_column = 'rt'

# Initialize a dictionary to store the best models and their performance for each stock
SVR_performance = {}
SVR_stock_return={}
# Loop over each stock
for stock_name, windows in ESG_Train_Test_Dict.items():
    stock_best_models = []
    ret=[]
    # Loop over each pair of training/testing set
    years = [2017, 2018, 2019, 2020, 2021, 2022]
    for year, (train_df, test_df) in zip(years, windows):
        # Standardize the data
        scaler_X = StandardScaler()
        scaler_y = StandardScaler()
        X_train_scaled = scaler_X.fit_transform(train_df.iloc[:, 7:].drop(columns=[target_column]))
        y_train_scaled = scaler_y.fit_transform(train_df[[target_column]])
        X_test_scaled = scaler_X.transform(test_df.iloc[:, 7:].drop(columns=[target_column]))
        y_test_scaled = scaler_y.transform(test_df[[target_column]])

        # Flatten y arrays
        y_train_scaled = y_train_scaled.ravel()
        y_test_scaled = y_test_scaled.ravel()

        # Define and fit the SVR model
        svr_model = SVR(C=1.0, epsilon=0.1)
        svr_model.fit(X_train_scaled, y_train_scaled)

        # Predict on the test set
        y_pred_scaled = svr_model.predict(X_test_scaled)

        # Inverse tray_prensform predictions and actual values to original scale
        y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1)).flatten()
        y_test = scaler_y.inverse_transform(y_test_scaled.reshape(-1, 1)).flatten()

        ret.append(y_pred)


        # Calculate performance metrics
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(y_test, y_pred)

        # Calculate Hit Rate
        direction_pred = np.sign(np.diff(y_pred))
        direction_true = np.sign(np.diff(y_test))
        hit_rate = np.mean(direction_pred == direction_true)

        # Store the best model and its performance metrics
        stock_best_models.append({
            'model': svr_model,
            'mse': mse,
            'rmse': rmse,
            'mae': mae,
            'hit_rate': hit_rate
        })

        print(f"Prediction for year {year} of {stock_name} is done.")
        print(f"{year}: {stock_best_models[-1]}")

    # Add the best models and their performance to the dictionary
    SVR_performance[stock_name] = stock_best_models
    SVR_stock_return[stock_name] = ret
end_time = time.time()
print(f"Execution time: {end_time - start_time} seconds")

Prediction for year 2017 of COP is done.
2017: {'model': SVR(), 'mse': 3.3729849823396476e-06, 'rmse': 0.0018365688068623096, 'mae': 0.001043850166659206, 'hit_rate': 0.98}
Prediction for year 2018 of COP is done.
2018: {'model': SVR(), 'mse': 2.0141656559117565e-05, 'rmse': 0.004487945694760306, 'mae': 0.002046322299664488, 'hit_rate': 0.964}
Prediction for year 2019 of COP is done.
2019: {'model': SVR(), 'mse': 0.0012108547766182009, 'rmse': 0.03479733864274969, 'mae': 0.010414360167740663, 'hit_rate': 0.9362549800796812}
Prediction for year 2020 of COP is done.
2020: {'model': SVR(), 'mse': 0.0004282426826441104, 'rmse': 0.02069402528857328, 'mae': 0.011031845001238931, 'hit_rate': 0.924}
Prediction for year 2021 of COP is done.
2021: {'model': SVR(), 'mse': 8.034370563972404e-06, 'rmse': 0.0028344965274228868, 'mae': 0.001949429932150481, 'hit_rate': 0.9723320158102767}
Prediction for year 2022 of COP is done.
2022: {'model': SVR(), 'mse': 6.889662782501557e-05, 'rmse': 0.008300399

In [20]:
start_time = time.time()
target_column = 'rt'

# Initialize a dictionary to store the best models and their performance for each stock
BPNN_performance = {}
BPNN_stock_return = {}

# Loop over each stock
for stock_name, windows in ESG_Train_Test_Dict.items():
    stock_best_models = []
    ret = []

    # Loop over each pair of training/testing set
    years = [2017, 2018, 2019, 2020, 2021, 2022]
    for year, (train_df, test_df) in zip(years, windows):
        # Standardize the data
        scaler_X = StandardScaler()
        scaler_y = StandardScaler()
        X_train_scaled = scaler_X.fit_transform(train_df.iloc[:, 7:].drop(columns=[target_column]))
        y_train_scaled = scaler_y.fit_transform(train_df[[target_column]])
        X_test_scaled = scaler_X.transform(test_df.iloc[:, 7:].drop(columns=[target_column]))
        y_test_scaled = scaler_y.transform(test_df[[target_column]])

        # Flatten y arrays
        y_train_scaled = y_train_scaled.ravel()
        y_test_scaled = y_test_scaled.ravel()

        # Define and fit the BPNN model
        bpnn_model = Sequential()
        bpnn_model.add(Dense(10, input_dim=X_train_scaled.shape[1], activation='relu'))  # Adjust number of neurons
        bpnn_model.add(Dense(1, activation='linear'))
        bpnn_model.compile(loss='mean_squared_error', optimizer='adam')
        bpnn_model.fit(X_train_scaled, y_train_scaled, epochs=64, batch_size=32, verbose=0)  # Adjust epochs and batch_size

        # Predict on the test set
        y_pred_scaled = bpnn_model.predict(X_test_scaled)

        # Inverse transform predictions and actual values to original scale
        y_pred = scaler_y.inverse_transform(y_pred_scaled).flatten()
        y_test = scaler_y.inverse_transform(y_test_scaled.reshape(-1, 1)).flatten()

        ret.append(y_pred)

        # Calculate performance metrics
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(y_test, y_pred)

        # Calculate Hit Rate
        direction_pred = np.sign(np.diff(y_pred))
        direction_true = np.sign(np.diff(y_test))
        hit_rate = np.mean(direction_pred == direction_true)

        # Store the best model and its performance metrics
        stock_best_models.append({
            'model': bpnn_model,
            'mse': mse,

            'rmse': rmse,
            'mae': mae,
            'hit_rate': hit_rate
        })

        print(f"Prediction for year {year} of {stock_name} is done.")
        print(f"{year}: {stock_best_models[-1]}")

    # Add the best models and their performance to the dictionary
    BPNN_performance[stock_name] = stock_best_models
    BPNN_stock_return[stock_name] = ret

end_time = time.time()
print(f"Execution time: {end_time - start_time} seconds")


8/8 [==============================] - 0s 2ms/step
Prediction for year 2017 of COP is done.
2017: {'model': <keras.src.engine.sequential.Sequential object at 0x7bd2ba370610>, 'mse': 7.900472703400303e-06, 'rmse': 0.002810777953414375, 'mae': 0.0021215984065704537, 'hit_rate': 0.96}
8/8 [==============================] - 0s 1ms/step
Prediction for year 2018 of COP is done.
2018: {'model': <keras.src.engine.sequential.Sequential object at 0x7bd2b2ebaa10>, 'mse': 6.838561591855497e-06, 'rmse': 0.002615064357115422, 'mae': 0.0019015174336118605, 'hit_rate': 0.96}
8/8 [==============================] - 0s 1ms/step
Prediction for year 2019 of COP is done.
2019: {'model': <keras.src.engine.sequential.Sequential object at 0x7bd2ba351de0>, 'mse': 2.878198026020793e-05, 'rmse': 0.005364883993173378, 'mae': 0.0029175307216344846, 'hit_rate': 0.9561752988047809}
8/8 [==============================] - 0s 1ms/step
Prediction for year 2020 of COP is done.
2020: {'model': <keras.src.engine.sequential.

In [21]:
class ELMRegressor:
    def __init__(self, n_hidden_units):
        self.n_hidden_units = n_hidden_units

    def fit(self, X, y):
        self.input_weights = np.random.normal(size=[X.shape[1], self.n_hidden_units])
        self.biases = np.random.normal(size=[self.n_hidden_units])

        # Calculate hidden layer output
        H = np.tanh(np.dot(X, self.input_weights) + self.biases)

        # Calculate output weights (beta)
        self.beta = np.dot(np.linalg.pinv(H), y)

    def predict(self, X):
        H = np.tanh(np.dot(X, self.input_weights) + self.biases)
        return np.dot(H, self.beta)
start_time = time.time()
target_column = 'rt'

# Initialize a dictionary to store the best models and their performance for each stock
ELM_performance = {}
ELM_stock_return={}
# Loop over each stock
for stock_name, windows in ESG_Train_Test_Dict.items():
    stock_best_models = []
    ret = []

    # Loop over each pair of training/testing set
    years = [2017, 2018, 2019, 2020, 2021, 2022]
    for year, (train_df, test_df) in zip(years, windows):
        # Standardize the data
        scaler_X = StandardScaler()
        scaler_y = StandardScaler()
        X_train_scaled = scaler_X.fit_transform(train_df.iloc[:, 7:].drop(columns=[target_column]))
        y_train_scaled = scaler_y.fit_transform(train_df[[target_column]])
        X_test_scaled = scaler_X.transform(test_df.iloc[:, 7:].drop(columns=[target_column]))
        y_test_scaled = scaler_y.transform(test_df[[target_column]])

        # Flatten y arrays
        y_train_scaled = y_train_scaled.ravel()
        y_test_scaled = y_test_scaled.ravel()

        # Define and fit the ELM model
        elm_model = ELMRegressor(n_hidden_units=100)
        elm_model.fit(X_train_scaled, y_train_scaled)

        # Predict on the test set
        y_pred_scaled = elm_model.predict(X_test_scaled)

        # Inverse transform predictions and actual values to original scale
        y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1)).flatten()
        y_test = scaler_y.inverse_transform(y_test_scaled.reshape(-1, 1)).flatten()

        ret.append(y_pred)
        # Calculate performance metrics
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(y_test, y_pred)

        # Calculate Hit Rate
        direction_pred = np.sign(np.diff(y_pred))
        direction_true = np.sign(np.diff(y_test))
        hit_rate = np.mean(direction_pred == direction_true)

        # Store the best model and its performance metrics
        stock_best_models.append({
            'model': elm_model,
            'mse': mse,
            'rmse': rmse,
            'mae': mae,
            'hit_rate': hit_rate
        })

        print(f"Prediction for year {year} of {stock_name} is done.")
        print(f"{year}: {stock_best_models[-1]}")

    # Add the best models and their performance to the dictionary
    ELM_performance[stock_name] = stock_best_models
    ELM_stock_return[stock_name] = ret
end_time = time.time()
print(f"Execution time: {end_time - start_time} seconds")

Prediction for year 2017 of COP is done.
2017: {'model': <__main__.ELMRegressor object at 0x7bd1ed7aabf0>, 'mse': 2.9267821083478094e-05, 'rmse': 0.005409974222071496, 'mae': 0.004171813129202302, 'hit_rate': 0.912}
Prediction for year 2018 of COP is done.
2018: {'model': <__main__.ELMRegressor object at 0x7bd2b87d3070>, 'mse': 3.739652846403553e-05, 'rmse': 0.006115270105566518, 'mae': 0.0048355916598162236, 'hit_rate': 0.916}
Prediction for year 2019 of COP is done.
2019: {'model': <__main__.ELMRegressor object at 0x7bd1ed7fb6d0>, 'mse': 0.0008362101106291931, 'rmse': 0.0289172977753661, 'mae': 0.0101574667993524, 'hit_rate': 0.9243027888446215}
Prediction for year 2020 of COP is done.
2020: {'model': <__main__.ELMRegressor object at 0x7bd1ed7aab60>, 'mse': 0.0003291341058035321, 'rmse': 0.018142053516719988, 'mae': 0.013541624404073034, 'hit_rate': 0.876}
Prediction for year 2021 of COP is done.
2021: {'model': <__main__.ELMRegressor object at 0x7bd20796c670>, 'mse': 0.0001136061196

In [22]:
def calculate_average_metrics(data):
    averages = {}
    for stock, records in data.items():
        # 初始化累计指标
        total_metrics = {'mse': 0, 'rmse': 0, 'mae': 0, 'hit_rate': 0}

        # 累加每条记录的指标
        for record in records:
            total_metrics['mse'] += record['mse']
            total_metrics['rmse'] += record['rmse']
            total_metrics['mae'] += record['mae']
            total_metrics['hit_rate'] += record['hit_rate']

        # 计算平均值
        averages[stock] = {metric: total / len(records) for metric, total in total_metrics.items()}

    return averages

def calculate_average_by_year(data):
    year_metrics = {2017: {'mse': 0, 'rmse': 0, 'mae': 0, 'hit_rate': 0},
                    2018: {'mse': 0, 'rmse': 0, 'mae': 0, 'hit_rate': 0},
                    2019: {'mse': 0, 'rmse': 0, 'mae': 0, 'hit_rate': 0},
                    2020: {'mse': 0, 'rmse': 0, 'mae': 0, 'hit_rate': 0},
                    2021: {'mse': 0, 'rmse': 0, 'mae': 0, 'hit_rate': 0},
                    2022: {'mse': 0, 'rmse': 0, 'mae': 0, 'hit_rate': 0}}
    counts = {year: 0 for year in year_metrics}
    for stock, records in data.items():
        for index, record in enumerate(records):
            year = 2017 + index
            # 累加指标
            year_metrics[year]['mse'] += record['mse']
            year_metrics[year]['rmse'] += record['rmse']
            year_metrics[year]['mae'] += record['mae']
            year_metrics[year]['hit_rate'] += record['hit_rate']
            counts[year] += 1
    for year in year_metrics:
        for metric in year_metrics[year]:
            year_metrics[year][metric] /= counts[year]

    return year_metrics
# calculate mean
RF_averages = calculate_averages(RF_performance)
rf=pd.DataFrame(RF_averages)
rf["model"]="rf"
LSTM_averages = calculate_average_by_year(LSTM_performance)
lstm=pd.DataFrame(LSTM_averages)
lstm["model"]="lstm"
SVR_averages = calculate_average_by_year(SVR_performance)
svr=pd.DataFrame(SVR_averages)
svr["model"]="svr"
BPNN_averages = calculate_average_by_year(BPNN_performance)
bpnn=pd.DataFrame(BPNN_averages)
bpnn["model"]="bpnn"
ELM_averages = calculate_average_by_year(ELM_performance)
elm=pd.DataFrame(ELM_averages)
elm["model"]="elm"

In [23]:
import numpy as np

# Number of simulations
num_simulations = 100

# Initialize a dictionary to store all predictions for each stock
all_predictions = {stock_name: [] for stock_name in ESG_Train_Test_Dict.keys()}

# Monte Carlo simulation: repeat the prediction process 1000 times
for _ in range(num_simulations):
    for stock_name, windows in ESG_Train_Test_Dict.items():
        # Select the most recent testing set as the future data to predict
        _, most_recent_test_df = windows[-1]  # Last pair in the list is the most recent

        # Standardize the feature columns of the most recent data
        X_future_scaled = scaler_X.transform(most_recent_test_df.iloc[:, 7:].drop(columns=[target_column]))

        # Use the last trained BPNN model for this stock to make predictions on the future data
        last_model = BPNN_performance[stock_name][-1]['model']  # Last model in the list for the stock
        future_pred_scaled = last_model.predict(X_future_scaled)

        # Inverse transform the predictions to the original scale
        future_pred = scaler_y.inverse_transform(future_pred_scaled).flatten()

        # Store the predictions
        all_predictions[stock_name].append(future_pred)

8/8 [==============================] - 0s 2ms/step


In [24]:
mean_future_returns = {}
for stock_name, preds in all_predictions.items():
  lst=[]
  for pred in preds:
    re=1
    for i in pred:
      re*=(1+i)
    lst.append(re)
  mean_future_returns[stock_name] = np.mean(lst)
# Calculate the mean predicted return for each stock across all simulations


# Sort the stocks by their mean predicted return in descending order
sorted_stocks_by_return = sorted(mean_future_returns.items(), key=lambda x: x[1], reverse=True)

# Select the top eight stocks
top_eight_stocks = sorted_stocks_by_return[:8]

# Extract just the stock names if needed
top_eight_stock_names = [stock[0] for stock in top_eight_stocks]

print("Top eight stocks based on mean predicted future returns across 1000 simulations:")
for stock, mean_return in top_eight_stocks:
    print(f"{stock}: {mean_return:.4f}")

Top eight stocks based on mean predicted future returns across 1000 simulations:
EPAM: 1.3881
MPC: 1.3677
DECK: 1.3162
LECO: 1.3061
LNG: 1.1789
KBR: 1.1143
HES: 1.1072
CDNS: 1.1065


In [25]:
for stock_name, preds in all_predictions.items():
  lst=[]
  for pred in preds:
    re=1
    for i in pred:
      re*=(1+i)
    lst.append(re)
  mean_future_returns[stock_name] = np.mean(lst)

In [26]:
mean_returns=[]
stocks=[]
for stock, ret in top_eight_stocks:
  mean_returns.append(ret)
  stocks.append(stock)

In [27]:
future_return=pd.DataFrame()
for i in stocks:
  future_return[i]=np.mean(all_predictions[i], axis=0)
future_return

,EPAM,MPC,DECK,LECO,LNG,KBR,HES,CDNS
0,0.017666,0.005254,-0.003503,-0.004576,0.025423,0.020110,0.014824,0.013817
1,0.091642,0.020945,-0.019387,-0.002869,0.012668,-0.007925,0.042350,-0.034625
2,-0.013869,-0.028533,-0.019940,-0.020453,-0.042469,-0.020283,-0.010944,-0.028931
3,-0.003823,0.020731,0.027162,0.002646,-0.007919,0.001345,0.014049,0.001447
4,0.023231,0.024716,0.025811,0.014273,-0.012304,0.009239,0.011802,0.011888
...,...,...,...,...,...,...,...,...
246,0.021492,0.008562,0.003316,0.009550,0.007256,0.011100,-0.005640,0.012018
247,-0.005277,0.014616,0.008941,0.000969,0.007753,0.011876,0.071398,0.004620
248,-0.020177,-0.077226,0.004009,-0.037888,-0.026400,-0.011418,-0.018012,0.003050
249,-0.029757,0.046956,-0.016645,-0.023643,0.020109,-0.007900,-0.003561,-0.021588


In [28]:
cov_matrix = future_return.cov()
# Number of assets
num_assets = len(mean_returns)

# Objective function: Portfolio volatility
def portfolio_volatility(weights, mean_returns, cov_matrix):
    return np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))

# Constraints: Weights sum to 1
constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})

# Bounds for weights
bounds = tuple((0, 1) for asset in range(num_assets))

# Initial guess for weights
initial_guess = num_assets * [1. / num_assets,]

# Optimization function
def minimize_volatility(mean_returns, cov_matrix):
    opt_result = minimize(fun=portfolio_volatility,
                          x0=initial_guess,
                          args=(mean_returns, cov_matrix),
                          method='SLSQP',
                          bounds=bounds,
                          constraints=constraints)
    return opt_result.x

# Calculate optimal weights
optimal_weights = minimize_volatility(mean_returns, cov_matrix)

for stock, weight in zip(stocks, optimal_weights):
    print(f"{stock}: {weight:.20%}")

EPAM: 1.52373167774219653303%
MPC: 5.62834205040087542926%
DECK: 0.00000000000000000000%
LECO: 39.45709988422423464272%
LNG: 4.06386463050811563846%
KBR: 44.46939613683644410003%
HES: 0.00000000000000038489%
CDNS: 4.85756562028814187215%


In [29]:
sum(optimal_weights*mean_returns)

1.2106859508926764